# Visão Geral
---

## Usando um Jupyter Notebook

Para esta parte do tutorial Mythical Mysfits, você estará interagindo diretamente com este Jupyter notebook para construir um modelo de machine learning usando dados de amostra que nós fornecemos. Um notebook oferece a capacidade de criar uma documentação rica ao lado do código e de sua saída, ajudando a descrever e executar as etapas realizadas em direção aos seus objetivos de aprendizado de máquina em um único local.

Cada parte independentemente executável de um notebook é representada como uma **célula** distinta. Uma célula pode ser de um dos três tipos:
* Markdown (rich text)
* Code (Python nesse notebook)
* Raw (conteúdos usados ​​diretamente da saída da própria célula, nota usada neste caderno)

Se você clicar em torno deste documento de notebook em várias partes dele, você verá uma borda realçar uma parte do documento, que representa uma célula.

Selecionar uma célula e clicar no botão **Run** na barra de ferramentas executará essa célula, assim como pressionar Ctrl+Enter no teclado. Para uma célula Markdown, isso formatará o texto escrito e o exibirá de acordo com a sintaxe de Markdown. Para uma célula Code, o código Python será executado no kernel subjacente e a saída do código será exibida sob a célula.

Para as células Code, você pode notar `In []:` à direita de cada célula. Isso é usado para indicar o status de execução e a seqüência de cada bloco de código dentro do notebook. Os colchetes vazios (`[]`) indicam que um bloco de código ainda possui uma nota sendo executada. Quando um bloco de código está no meio de sua execução, mas ainda não a completou, você verá `[*]` ser exibido. E finalmente, uma vez que um bloco de código tenha terminado a execução, você verá um número específico exibido como `[1]`. Este número representa a seqüência em que esse bloco de código foi executado em relação àqueles antes dele dentro do notebook. Isso é para ajudá-lo a acompanhar o estado atual da execução do código na totalidade do documento do notebook (como você pode executar uma célula e, em seguida, ler alguma documentação e não lembrar exatamente qual célula foi executada pela última vez!).

Se você precisar reverter o processamento por qualquer motivo, você pode usar o menu ** Kernel ** acima na barra de ferramentas do notebook para resetar o kernel, limpar o output, etc.

## O Notebook de Recomendações de Mysfits

O código necessário para usar os dados de amostra e criar um modelo de machine learning já foi escrito e está contido nas seguintes células abaixo neste notebook. É sua tarefa ler a documentação para entender as etapas realizadas e familiarizar-se com a interação com este notebook para organizar dados, construir e treinar o modelo de machine learning e implantar esse modelo para uso de nossa aplicação.

# Parte 1: Fazendo o Download dos Dados de Amostra
---
A célula de código abaixo faz o download dos dados de amostra que foram testados no S3.

O conjunto de dados contém as respostas a um questionário de quase um milhão de usuários imaginários do site Mythical Mysfits e qual é o seu Mysfit favorito. Para casos de uso como este, em que o algoritmo utilizado espera entradas numéricas, mapeamos cada possível resposta do questionário e o mysfit escolhido para um valor numérico. O resultado do questionário de cinco perguntas e um mysfit favorito é um arquivo CSV onde cada linha contém 6 valores separados por vírgula (Exemplo: `1,0,2,7,0,11`).  Por favor visite o [Website do Mythical Mysfits](http://www.mythicalmysfits.com) para testar o questionário você mesmo.

Clique na célula de código abaixo para delimitá-la e, em seguida, clique em **Run** acima na barra de ferramentas ou pressione Ctrl+Enter no seu teclado para baixar o conjunto de dados de amostra e armazená-lo no diretório listado.

In [ ]:
%%bash

wget 'https://s3.amazonaws.com/mysfit-recommendation-training-data/mysfit-preferences.csv.gz'
mkdir -p /tmp/mysfit/raw
mv mysfit-preferences.csv.gz /tmp/mysfit/raw/mysfit-preferences.csv.gz


# Parte 2: Preparação dos dados
---

## Pré-Processando os Dados
Agora que temos os dados brutos, vamos processá-los. 
Primeiro, carregaremos os dados em vetores numpy e os dividiremos aleatoriamente em treino e teste com uma divisão 90/10.

In [ ]:
import numpy as np
import os

data_dir = "/tmp/mysfit/"
processed_subdir = "standardized"
raw_data_file = os.path.join(data_dir, "raw", "mysfit-preferences.csv.gz")
train_features_file = os.path.join(data_dir, processed_subdir, "train/csv/features.csv")
train_labels_file = os.path.join(data_dir, processed_subdir, "train/csv/labels.csv")
test_features_file = os.path.join(data_dir, processed_subdir, "test/csv/features.csv")
test_labels_file = os.path.join(data_dir, processed_subdir, "test/csv/labels.csv")

# read raw data
print("Reading raw data from {}".format(raw_data_file))
raw = np.loadtxt(raw_data_file, delimiter=',')

# split into train/test with a 90/10 split
np.random.seed(0)
np.random.shuffle(raw)
train_size = int(0.9 * raw.shape[0])
train_features = raw[:train_size, :-1]
train_labels = raw[:train_size, -1]
test_features = raw[train_size:, :-1]
test_labels = raw[train_size:, -1]

## Fazendo o upload para o Amazon S3
Agora, como normalmente o conjunto de dados será grande e está localizado no Amazon S3, vamos gravar os dados no Amazon S3 no formato recordio-protobuf. Primeiro, criamos um buffer de io envolvendo os dados, em seguida fazemos o upload para o Amazon S3. Observe que a escolha do bucket e prefixo deve mudar para usuários diferentes e conjuntos de dados diferentes

In [ ]:
import io
import sagemaker.amazon.common as smac

print('train_features shape = ', train_features.shape)
print('train_labels shape = ', train_labels.shape)

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, train_features, train_labels)
buf.seek(0)

In [ ]:
import boto3
import os
import sagemaker

bucket = sagemaker.Session().default_bucket() # modify to your bucket name
prefix = 'mysfit-recommendation-dataset'
key = 'recordio-pb-data'

boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))


Também é possível fornecer dados de teste. Dessa forma, podemos obter uma avaliação do desempenho do modelo a partir dos logs de treinamento. Para usar esse recurso, vamos fazer o upload dos dados de teste para o Amazon S3 também

In [ ]:
print('test_features shape = ', test_features.shape)
print('test_labels shape = ', test_labels.shape)

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, test_features, test_labels)
buf.seek(0)

boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test', key)).upload_fileobj(buf)
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, prefix, key)
print('uploaded test data location: {}'.format(s3_test_data))


# Parte 3: Treinamento
---

Levamos um momento para explicar em alto nível como treinamento e previsão de Machine Learning funcionam no Amazon SageMaker. Primeiro, precisamos treinar um modelo. Este é um processo que, dado um conjunto de dados e hiperparâmetros rotulados, guiando o processo de treinamento, gera um modelo. Uma vez que o treinamento é feito, nós configuramos o que é chamado de ** endpoint **. Um endpoint é um serviço web que, dada uma requisição contendo um datapoint não rotulado, ou mini-batch de datapoint, retorna uma predição (ou predições).

No Amazon SageMaker, o treinamento é feito através de um objeto chamado **estimator**. Ao configurar o estimator, especificamos a localização (no Amazon S3) dos dados de treinamento, o caminho (novamente no Amazon S3) para o diretório de saída onde o modelo será serializado, hiperparâmetros genéricos como o tipo de máquina a ser usado durante o processo de treinamento, e hiperparâmetros específicos para kNN, como o tipo de índice, etc. Uma vez que o estimator é inicializado, podemos chamar o seu método **fit** para fazer o treinamento real.

Agora que estamos prontos para o treinamento, começamos com uma função de conveniência que inicia um job de treinamento.

In [ ]:
import matplotlib.pyplot as plt

import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer, json_deserializer
from sagemaker.amazon.amazon_estimator import get_image_uri


def trained_estimator_from_hyperparams(s3_train_data, hyperparams, output_path, s3_test_data=None):
    """
    Create an Estimator from the given hyperparams, fit to training data, 
    and return a deployed predictor
    
    """
    # set up the estimator
    knn = sagemaker.estimator.Estimator(get_image_uri(boto3.Session().region_name, "knn"),
        get_execution_role(),
        train_instance_count=1,
        train_instance_type='ml.m5.2xlarge',
        output_path=output_path,
        sagemaker_session=sagemaker.Session())
    knn.set_hyperparameters(**hyperparams)
    
    # train a model. fit_input contains the locations of the train and test data
    fit_input = {'train': s3_train_data}
    if s3_test_data is not None:
        fit_input['test'] = s3_test_data
    knn.fit(fit_input)
    return knn

Agora, nós executamos o job real de treinamento. Por enquanto, nos atemos aos parâmetros padrão.

In [ ]:
hyperparams = {
    'feature_dim': 5,
    'k': 10,
    'sample_size': 100000,
    'predictor_type': 'classifier' 
}
output_path = 's3://' + bucket + '/' + prefix + '/default_example/output'
knn_estimator = trained_estimator_from_hyperparams(s3_train_data, hyperparams, output_path, 
                                                   s3_test_data=s3_test_data)

Observe que mencionamos um conjunto de testes no job de treinamento. Quando um conjunto de testes é fornecido, o job de treinamento não apenas produz um modelo, mas também o aplica ao conjunto de testes e informa a precisão. Nos logs, você pode ver a precisão do modelo no conjunto de testes.

# Parte 4: Implantando o modelo em um Endpoint do SageMaker 
---

## Configurando o endpoint

Agora que temos um modelo treinado, estamos prontos para executar a inferência. O objeto **knn_estimator** acima contém todas as informações necessárias para hospedar o modelo. Abaixo, fornecemos uma função de conveniência que, dado um estimador, configura o endpoint que hospeda o modelo. Além do objeto estimador, fornecemos um nome (string) para o estimador e um **instance_type**. O **instance_type** é o tipo de máquina que hospedará o modelo. Ele não é restrito de forma alguma pelas configurações de parâmetros do job de treinamento.

In [ ]:
def predictor_from_estimator(knn_estimator, estimator_name, instance_type, endpoint_name=None): 
    knn_predictor = knn_estimator.deploy(initial_instance_count=1, instance_type=instance_type,
                                        endpoint_name=endpoint_name)
    knn_predictor.content_type = 'text/csv'
    knn_predictor.serializer = csv_serializer
    knn_predictor.deserializer = json_deserializer
    return knn_predictor

In [ ]:
import time

instance_type = 'ml.m4.xlarge'
model_name = 'knn_%s'% instance_type
endpoint_name = 'knn-ml-m4-xlarge-%s'% (str(time.time()).replace('.','-'))
print('setting up the endpoint..')
predictor = predictor_from_estimator(knn_estimator, model_name, instance_type, endpoint_name=endpoint_name)

## Inferência

Agora que temos nosso preditor, vamos usá-lo em nosso conjunto de dados de teste. O código a seguir é executado no conjunto de dados de teste, calcula a precisão e a latência média. Divide os dados em 100 batches. Em seguida, cada batch é fornecido ao serviço de inferência para obter predições. Assim que tivermos todas as predições, calculamos a sua precisão, dados os rótulos verdadeiros do conjunto de testes.

In [ ]:

batches = np.array_split(test_features, 100)
print('data split into 100 batches, of size %d.' % batches[0].shape[0])
# obtain an np array with the predictions for the entire test set
start_time = time.time()
predictions = []
for batch in batches:
    result = predictor.predict(batch)
    cur_predictions = np.array([result['predictions'][i]['predicted_label'] for i in range(len(result['predictions']))])
    predictions.append(cur_predictions)
predictions = np.concatenate(predictions)
run_time = time.time() - start_time

test_size = test_labels.shape[0]
num_correct = sum(predictions == test_labels)
accuracy = num_correct / float(test_size)
print('time required for predicting %d data point: %.2f seconds' % (test_size, run_time))
print('accuracy of model: %.1f%%' % (accuracy * 100) )

**Nota**: Lembre-se de que esse conjunto de dados de amostra foi gerado aleatoriamente. Portanto, você notará a baixa precisão que esse modelo é capaz de alcançar (porque há pouco padrão nos dados que estão sendo usados ​​para criar o modelo). 

Para os seus casos de uso futuros usando o aprendizado de máquina e o SageMaker, cabe a você determinar o nível de precisão exigido para que o modelo seja benéfico para sua aplicação. Nem todos os casos de uso exigem mais de 90% de precisão para que os benefícios sejam obtidos. Embora, para alguns casos de uso, especialmente quando a segurança do cliente fizer parte de sua aplicação, você pode determinar que um modelo deve ter níveis extremos de precisão para ser utilizado em Produção.

# PARE!

## Próximos Passos do Workshop Mythical Mysfits 
Você acaba de implantar um endpoint de predição para o SageMaker. Ele pode ser invocado via HTTP diretamente. No entanto, em vez de diretamente integrar o frontend de nossa aplicação com o endpoint nativo do SageMaker, vamos incluir nossa própria API RESTful e serverless em neste endpoint de previsão. Por favor retorne às instruções do workshop e prossiga para o próximo passo para continuar o tutorial. 

***

---
# Clean-Up quando Completar o Módulo 7

## Deletando o endpoint

Agora terminamos com o exemplo, exceto um ato final de clean-up. Ao configurar o endpoint, iniciamos uma máquina na nuvem e, enquanto não for excluída, a máquina ainda estará funcionando e estamos pagando por ela. Uma vez que o endpoint não é mais necessário, nós o deletamos. O código a seguir faz exatamente isso.

In [ ]:
def delete_endpoint(predictor):
    try:
        boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
        print('Deleted {}'.format(predictor.endpoint))
    except:
        print('Already deleted: {}'.format(predictor.endpoint))

delete_endpoint(predictor)
            